# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
## Encode 'homeowner' status 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [5]:
# homeowner column
le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,1,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,1,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,2,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,1,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,0,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status'].to_frame()

In [7]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [8]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [9]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(58152, 8)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

## Gives error saying : " could not convert string to float: 'mortgage' "
## Go back above and encode 'homeowner' data/status 
## Now that its encoded, it works 

In [12]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
BAS_LR = balanced_accuracy_score(y_test, y_pred)
BAS_LR

0.9543211898288821

In [15]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]], dtype=int64)

In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

cr_imb = classification_report_imbalanced(y_test, y_pred,output_dict=True)
recall_LR = cr_imb['avg_rec']
geo_LR = cr_imb['avg_geo']
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [17]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({'loan_status': 1})

In [18]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [19]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = model.predict(X_test)
BAS_OS = balanced_accuracy_score(y_test, y_pred)
BAS_OS

0.9948013433551894

In [20]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  105, 18654]], dtype=int64)

In [21]:
# Print the imbalanced classification report
# YOUR CODE HERE

cr_imb = classification_report_imbalanced(y_test, y_pred,output_dict=True)
recall_OS = cr_imb['avg_rec']
geo_OS = cr_imb['avg_geo']
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [22]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({'loan_status': 1})

In [23]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = model.predict(X_test)
BAS_SMOTE = balanced_accuracy_score(y_test, y_pred)
BAS_SMOTE

0.9948279972279972

In [25]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  104, 18655]], dtype=int64)

In [26]:
# Print the imbalanced classification report
# YOUR CODE HERE
cr_imb = classification_report_imbalanced(y_test, y_pred,output_dict=True)
recall_SMOTE = cr_imb['avg_rec']
geo_SMOTE = cr_imb['avg_geo']
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [27]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({'loan_status': 1})

In [28]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [29]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred = model.predict(X_test)
BAS_US = balanced_accuracy_score(y_test, y_pred)
BAS_US

0.9836813049736126

In [30]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[  608,    17],
       [  102, 18657]], dtype=int64)

In [31]:
# Print the imbalanced classification report
# YOUR CODE HERE
cr_imb = classification_report_imbalanced(y_test, y_pred,output_dict=True)
recall_US = cr_imb['avg_rec']
geo_US = cr_imb['avg_geo']
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.97      0.99      0.91      0.98      0.97       625
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18759

avg / total       0.99      0.99      0.97      0.99      0.98      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [32]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({'loan_status': 1})

In [33]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [34]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred = model.predict(X_test)
BAS_SMTN = balanced_accuracy_score(y_test, y_pred)
BAS_SMTN

0.994748035609574

In [35]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  107, 18652]], dtype=int64)

In [36]:
# Print the imbalanced classification report
# YOUR CODE HERE

cr_imb = classification_report_imbalanced(y_test, y_pred,output_dict=True)
recall_SMTN = cr_imb['avg_rec']
geo_SMTN = cr_imb['avg_geo']
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   The SMOTE oversample method ended up having just barely the best accuracy score. However Random Oversampling 
   and Combination Sampling through SMOTTEEN also acheived higher than 99% accuracy, as well. 

2. Which model had the best recall score?

    SMOTE again, but also just barely. 

3. Which model had the best geometric mean score?

    SMOTE again, but also just barely. 


In [37]:
print(BAS_LR)
print(BAS_OS)
print(BAS_SMOTE)
print(BAS_US)
print(BAS_SMTN)
BAS_list = [BAS_LR,BAS_OS,BAS_SMOTE,BAS_US,BAS_SMTN]
BAS_list.sort(reverse=True)
BAS_list
# SMOTE 

0.9543211898288821
0.9948013433551894
0.9948279972279972
0.9836813049736126
0.994748035609574


[0.9948279972279972,
 0.9948013433551894,
 0.994748035609574,
 0.9836813049736126,
 0.9543211898288821]

In [38]:
print(recall_LR)
print(recall_OS)
print(recall_SMOTE)
print(recall_US)
print(recall_SMTN)
recall_list = [recall_LR,recall_OS,recall_SMOTE,recall_US,recall_SMTN]
recall_list.sort(reverse=True)
recall_list
# SMOTE 

0.9924164259182832
0.994428394552208
0.9944799834915394
0.9938609162195625
0.9943252166735452


[0.9944799834915394,
 0.994428394552208,
 0.9943252166735452,
 0.9938609162195625,
 0.9924164259182832]

In [39]:
print(geo_LR)
print(geo_OS)
print(geo_SMOTE)
print(geo_US)
print(geo_SMTN)
geo_list = [geo_LR,geo_OS,geo_SMOTE,geo_US,geo_SMTN]
geo_list.sort(reverse=True)
geo_list
# SMOTE 

0.9534520009184173
0.9948012634763633
0.9948279276752365
0.9836211196170307
0.9947479329344174


[0.9948279276752365,
 0.9948012634763633,
 0.9947479329344174,
 0.9836211196170307,
 0.9534520009184173]